In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import random
import glob
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

In [4]:
MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"

In [5]:
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
bert_sc = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [9]:
text_list = [
"この映画は面白かった。",
"この映画の最後にはがっかりさせられた。",
"この映画を見て幸せな気持ちになった。"
]

label_list = [1, 0, 1]
encoding = tokenizer(text_list, padding="longest", return_tensors="pt")
labels = torch.tensor(label_list)

with torch.no_grad():
    output = bert_sc.forward(**encoding)
scores = output.logits
labels_predicted = scores.argmax(-1)
num_correct = (labels_predicted==labels).sum().item()
accuracy = num_correct / labels.size(0)

print("# scores size: ")
print(scores.size())
print("# predicted labels: ")
print(labels_predicted)
print("# accuracy")
print(accuracy)


# scores size: 
torch.Size([3, 2])
# predicted labels: 
tensor([1, 1, 1])
# accuracy
0.6666666666666666


In [16]:
# calculate loss
encoding = tokenizer(text_list, padding="longest", return_tensors="pt")
encoding['labels'] = torch.tensor(label_list)
output = bert_sc(**encoding)
print(output.loss)

tensor(0.7102, grad_fn=<NllLossBackward0>)
